## A simple perceptron algorithm

In [1]:

import random
import numpy as np
import scipy.io as sio
import plotly.plotly as py
from plotly.graph_objs import *

data = sio.loadmat('datasets/dataset1.mat')

w_gen_feas = (data['w_gen_feas']).transpose()
pos_examples_nobias = data['pos_examples_nobias']
neg_examples_nobias = data['neg_examples_nobias']
w_init = (data['w_init']).transpose()

learn_perceptron(neg_examples_nobias,pos_examples_nobias,w_init,w_gen_feas)


In [2]:
def learn_perceptron(neg_examples_nobias,pos_examples_nobias,w_init,w_gen_feas):
    num_neg_examples = (neg_examples_nobias).shape
    num_pos_examples = (pos_examples_nobias).shape
    num_errors_history = []
    
    #Adding a column of ones to learn the bias parameters. 
    
    neg_examples = np.ones([num_neg_examples[0], num_neg_examples[1]+1])
    neg_examples[:,:-1] = neg_examples_nobias
    pos_examples = np.ones([num_pos_examples[0], num_pos_examples[1]+1])
    pos_examples[:,:-1] = pos_examples_nobias
    
    #print neg_examples_nobias
    #print neg_examples
    
    #neg_examples = [x + [1] for x in neg_examples_nobias]
    #pos_examples = [x + [1] for x in pos_examples_nobias]
    
    if w_init is None: 
        w = np.ndarray([random.random() for i in range(3)])
    else: 
        w = w_init
     
    if w_gen_feas is None:
        w_gen_feas = np.zeros(shape=w.shape)
        
    
    
    neg_example_mistakes, pos_example_mistakes = eval_perceptron(neg_examples, pos_examples, w) 
    total_errors = len(neg_example_mistakes) + len(pos_example_mistakes)
    num_errors_history.append(total_errors)
    
    while total_errors > 0:
        
        w = update_weights(neg_examples, pos_examples, w)
        neg_example_mistakes, pos_example_mistakes = eval_perceptron(neg_examples, pos_examples, w) 
        total_errors = len(neg_example_mistakes) + len(pos_example_mistakes)
        num_errors_history.append(total_errors)
        print "Number of errors : {}".format(total_errors)
        print "Weights: "
        print w 
        
        key = raw_input('Press enter to continue, q to quit:')
        if key is 'q':
            break


In [3]:
def update_weights(neg_examples, pos_examples, w_current): 
    
    w = w_current
    
    for i in xrange(0, len(neg_examples)):
        this_case = neg_examples[i][:]
        x = (np.array(this_case)).transpose()
        
        activation = np.sum(np.array(this_case) * np.array(w))
        if activation >= 0: 
            w = (np.array(w) - np.array(x))
    
    for i in xrange(0, len(pos_examples)):
        this_case = pos_examples[i][:]
        x = (np.array(this_case)).transpose()
        
        activation = np.sum(np.array(this_case) * np.array(w))
        if activation < 0: 
            w = (np.array(w) + np.array(x))
            
    return w


In [4]:
def eval_perceptron(neg_examples, pos_examples, w): 
    neg_example_mistakes = [];
    pos_example_mistakes = [];
    
    for i in xrange(0,len(neg_examples)): 
        x = (neg_examples[i][:]).transpose()
        #print "printing x"
        #print x
        #break
        activation = np.sum((np.array(x)).transpose() * np.array(w))
        #print activation
        if activation >= 0:
            neg_example_mistakes.append(i)
            
    for i in xrange(0,len(pos_examples)): 
        x = (pos_examples[i][:]).transpose()
        activation = np.sum((np.array(x)).transpose() * np.array(w))
        #print activation
        if activation < 0:
            pos_example_mistakes.append(i)
            
    return neg_example_mistakes, pos_example_mistakes
        

In [6]:
pos_trace = Scatter(
    x = pos_examples_nobias[:,0],
    y = pos_examples_nobias[:,1], 
    mode = 'markers',
    name = 'Positive Samples',
    marker = dict(
        size = 50,
        color = 'rgb(0,255,0)'
    )
)

neg_trace = Scatter(
    x = neg_examples_nobias[:,0],
    y = neg_examples_nobias[:,1], 
    mode = 'markers',
    name = 'Negative Samples',
    marker = dict(
        size = 50,
        color = 'rgb(255,0,0)'
    )
)

data = Data([pos_trace, neg_trace])
py.iplot(data, filename = 'basic-line')